In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'tensorflow'

In [2]:
!pip install tensorflow

  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.11.2
    Uninstalling Pygments-2.11.2:
      Successfully uninstalled Pygments-2.11.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.1
    Uninstalling protobuf-3.19.1:
      Successfully uninstalled protobuf-3.19.1
  Attempting uninstall: h5py
    Found existing installation: h5py 3.6.0
    Uninstalling h5py-3.6.0:
      Successfully uninstalled h5py-3.6.0
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.42.0
    Uninstalling grpcio-1.42.0:
      Successfully uninstalled grp

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 2.0.2 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 2.0.2 which is incompatible.
jupyter-server 1.13.5 requires pywinpty<2; os_name == "nt", but you have pywinpty 2.0.2 which is incompatible.


In [ ]:

# === 1. Simulate Vasicek Data ===
def simulate_vasicek(a, b, sigma, r0, T, dt=1.0):
    steps = int(T / dt)
    rates = [r0]
    for _ in range(steps):
        dr = a * (b - rates[-1]) * dt + sigma * np.random.normal() * np.sqrt(dt)
        rates.append(rates[-1] + dr)
    return np.array(rates)

true_a, true_b, true_sigma = 0.1, 0.03, 0.01
r0 = 0.025
T = 499  # 500 data points
rates = simulate_vasicek(true_a, true_b, true_sigma, r0, T)

# === 2. Preprocessing ===
scaler = MinMaxScaler()
rates_scaled = scaler.fit_transform(rates.reshape(-1, 1))

sequence_length = 50
X, y = [], []

for i in range(len(rates_scaled) - sequence_length):
    X.append(rates_scaled[i:i+sequence_length])
    y.append([true_a, true_b, true_sigma])  # Known parameters for supervision

X = np.array(X)
y = np.array(y)

# Reshape for LSTM: (samples, timesteps, features)
X = X.reshape((X.shape[0], X.shape[1], 1))

# === 3. Define the Model ===
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(sequence_length, 1)),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3)  # Output: a, b, sigma
])

model.compile(optimizer='adam', loss='mse')
model.summary()

# === 4. Train the Model ===
history = model.fit(X, y, epochs=500, verbose=0)

# === 5. Predict Parameters ===
predicted_params = model.predict(X)
final_pred = predicted_params[-1]

print("\n--- Final Predicted Parameters ---")
print(f"a:      {final_pred[0]:.5f}")
print(f"b:      {final_pred[1]:.5f}")
print(f"sigma:  {final_pred[2]:.5f}")

# === 6. Plot loss curve ===
plt.plot(history.history['loss'])
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE")
plt.show()